<a href="https://colab.research.google.com/github/rnanclarez/scripts_mestrado/blob/main/GCN_Modelo_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#%env CUDA_LAUNCH_BLOCKING=1
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

In [ ]:
#@title Teste de memória
### informação do tipo de processador usado(cpu/gpu...)
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

### informação de uso de memória
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
#@title setUp do ambiente
!python -c "import torch; print(torch.__version__)"

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

print(locale.getpreferredencoding())

!pip install transformers
!pip install torchdata

#ajustar a verão do pygeometric com a versão do python e cuda instalada
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
!pip install torch-geometric

#!pip install onnx


In [44]:
#@title Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

import torch.optim as optim
from torch.utils.data import Dataset #, DataLoader
from torch_geometric.data import Data
from torch_geometric.loader import NodeLoader , DataLoader

import pprint

from transformers import BertTokenizer, BertModel, BertForMaskedLM, get_constant_schedule_with_warmup, PreTrainedTokenizerFast

import pandas as pd
import numpy as np
import scipy.sparse as sps
from sklearn import preprocessing 
from sklearn.model_selection import KFold

import spacy
from spacy import displacy

import os
import gc 

import math
import time
import random

from sklearn.metrics import precision_recall_fscore_support, accuracy_score

#import torch.onnx



In [8]:
#@title Carregamento do Bert e do spacy

# Load pre-trained model (weights)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
nlp = spacy.load("en_core_web_sm")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [50]:
#@title Configurações { run: "auto", vertical-output: true, display-mode: "form" }
N_FOLDS = 10 #@param {type:"number"}

BATCH_SIZE = 8 #@param {type:"number"}
N_EPOCHS = 5 #@param {type:"number"}
warmup_percent = 0.2 #@param {type:"number"}
FOLDER = 'folds' #@param {type:"string"}
PREFIXO = 'coliee_data' #@param {type:"string"}
OUTPUT_DIM = 2 #@param {type:"number"}
device_form = "CUDA" #@param ["CPU", "CUDA"]

torch.use_deterministic_algorithms(True)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
device = torch.device('cuda' if torch.cuda.is_available() and device_form == 'CUDA' else 'cpu')
#device = torch.device('cpu')
criterion = nn.CrossEntropyLoss().to(device)

In [112]:
#@title Funções

def coo_matrix(texto):
  doc = nlp(texto)
  return zip(*[(child.head.i, child.i) for token in doc for child in token.children])

def join_sentences(sent1, sent2):
  doc = [tokenizer.cls_token.strip('[').strip(']')]
  doc.extend(tokenizer.tokenize(sent1))
  doc.append(tokenizer.sep_token.strip('[').strip(']'))
  doc.extend(tokenizer.tokenize(sent2))
  doc.append(tokenizer.sep_token.strip('[').strip(']'))
  return ' '.join(doc)

def calcularMetricas(preds, y, batch, batchSize):
  arg_max = preds.argmax(dim = 1).tolist()
  corretas = [ int(i[0] == i[1]) for i in zip(arg_max ,y.tolist())]
  erradas = [ int(i[0] != i[1]) for i in zip(arg_max ,y.tolist())]
  
  erros = [(i + batch*batchSize, arg_max[i], y.tolist()[i]) for i, j in enumerate(erradas)  if j == 1]

  #return correct.sum() / len(y)
  acc = accuracy_score(y.tolist(), arg_max)
  prf = precision_recall_fscore_support(y.tolist(), arg_max, average=None, labels=[0, 1])
  return acc, *prf, erros

def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

def getModelInputs(sent1, sent2):
  sent1 = sent1[:255] if len(sent1) >= 256 else sent1
  rows, cols = coo_matrix(join_sentences(sent1, sent2))
  edge_index = [rows, cols]
  
  pad_sequences = tokenizer(sent1, sent2, max_length=512, padding='max_length', truncation=True)
  
  sentences_join = pad_sequences['input_ids']
  masks_join =  pad_sequences['attention_mask']
  type_join = pad_sequences['token_type_ids']

  return  sentences_join, masks_join, type_join, edge_index

def getDataLoader(fold, batch, folder, prefixo, tipoDataSet):
  dataset = PygDataSet(f'{folder}/{prefixo}_{tipoDataSet}_{fold}.csv', 0)
  data_len = len(dataset)
  dataloader = DataLoader(dataset=dataset, batch_size=batch)

  return data_len, dataloader


def get_scheduler(optimizer, warmup_steps):
  return get_constant_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps)

def unzipBatch(batch, device):
  batch_size = len(batch)
  sequence = torch.stack(batch.x.transpose(0,1)[0].chunk(batch_size)).to(device) 
  attn_mask = torch.stack(batch.x.transpose(0,1)[1].chunk(batch_size)).to(device) 
  token_type = torch.stack(batch.x.transpose(0,1)[2].chunk(batch_size)).to(device) 
  edge_index = batch.edge_index.to(device)

  return sequence, attn_mask, token_type, edge_index

def treinarEpoca(model, iterator, optimizer, criterion, scheduler):
  epoch_loss = 0
  epoch_acc = 0
  model.train()
  for i, batch in enumerate(iterator):
    optimizer.zero_grad() # clear gradients first
    label = batch.y.type(torch.LongTensor).to(device)
    predictions = model(*unzipBatch(batch, device))

    loss = criterion(predictions, label)
    metricas = calcularMetricas(predictions, label, i, BATCH_SIZE)
    print(metricas[5])
    print("-----")
    loss.backward()
    optimizer.step()
    scheduler.step()
    epoch_loss += loss.detach().item()
    epoch_acc += metricas[0]
  return epoch_loss / len(iterator), epoch_acc / len(iterator)

def validarEpoca(model, iterator, criterion):
  epoch_loss = 0
  epoch_acc = 0
  model.eval()
  with torch.no_grad():
    for i, batch in enumerate(iterator):
      label = batch.y.type(torch.LongTensor).to(device)
      predictions = model(*unzipBatch(batch, device))

      loss = criterion(predictions, label)
      metricas = calcularMetricas(predictions, label, i, BATCH_SIZE)
      epoch_loss += loss.item()
      epoch_acc += metricas[0]
  return epoch_loss / len(iterator), epoch_acc / len(iterator)

def inferir(premissa, hipotese, model, classes):
  device = torch.device('cuda' if torch.cuda.is_available() and device_form == 'CUDA' else 'cpu')
  model.eval()
  
  le = preprocessing.LabelEncoder().fit(classes)

  inputs = getModelInputs(premissa, hipotese)

  prediction = model(torch.tensor(inputs[0])[None, :].to(device),
                      torch.tensor(inputs[1])[None, :].to(device),
                      torch.tensor(inputs[2])[None, :].to(device),
                      torch.tensor(inputs[3]).to(device))
  print(prediction)
  prediction = prediction.argmax(dim=-1).item()
  print(prediction)
  return le.inverse_transform([prediction])[0]


def salvarOnnx():

  valid_data_len,  valid_dataloader = getDataLoader(0, BATCH_SIZE, FOLDER, PREFIXO, 'valid')

  batch = next(iter(valid_dataloader))
  label = batch.y.type(torch.LongTensor).to(device)
  bert_model = BertModel.from_pretrained('bert-base-uncased')
  model = GCNBERTNLIModel(bert_model, OUTPUT_DIM).to(device)
  model.eval()
  inputs = unzipBatch(batch, device)
  predictions = model(inputs)
  torch.onnx.export(model, inputs, "model.onnx" , export_params=True, opset_version=10, do_constant_folding=True)


In [41]:
#@title Class PygDataSet
class PygDataSet(Dataset):
    def __init__(self, arquivo, skiprows=1):
        self.df = pd.read_csv(arquivo, skiprows = skiprows )
        self.df.dropna(axis=0, how='any', inplace=True) #remove linhas com colunas vazias
        self.le = preprocessing.LabelEncoder()
        self.le.fit(self.df.iloc[:,1])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sent1 = self.df.iloc[idx,2]
        sent2 = self.df.iloc[idx,3]

        sentences_join, masks_join, type_join, edge_index = getModelInputs(sent1, sent2)
        x = torch.tensor([sentences_join,masks_join,type_join]).transpose(0,1)
        indices = torch.tensor(edge_index)
        labels = self.le.transform([self.df.iloc[idx,1]])
        return  Data(edge_index=indices, x=x, y=torch.tensor(labels))
        


In [9]:
#%script false --no-raise-error
#@title carrega dados coliee

!cp -r /content/drive/MyDrive/mestrado/coliee_data/. ./coliee_data

task4df = pd.DataFrame( columns = ['label', 't1', 't2'])


# import required module
import os
# assign directory
directory = '/content/coliee_data/task3_4/COLIEE2021statute_data-English/train'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        task4df = pd.concat([task4df,pd.read_xml(f)])
#task4df.to_csv("coliee_data.csv")
kf = KFold(n_splits=10, shuffle=False)
!mkdir folds
for i, (t_ind, v_ind) in enumerate(kf.split(task4df)):
    
    #print(t_ind, v_ind)
    task4df.iloc[t_ind].to_csv(f'folds/coliee_data_train_{i}.csv')     # train set
    task4df.iloc[v_ind].to_csv(f'folds/coliee_data_valid_{i}.csv')    # validation set



In [ ]:
#@title carrega dados SNLI
#%%script false

df = pd.read_csv('/content/drive/MyDrive/mestrado/colab/snli_1.0/snli_1.0_train.txt', sep='\t')

df = pd.concat([df,pd.read_csv('/content/drive/MyDrive/mestrado/colab/snli_1.0/snli_1.0_dev.txt', sep='\t')])

df = pd.concat([df,pd.read_csv('/content/drive/MyDrive/mestrado/colab/snli_1.0/snli_1.0_test.txt', sep='\t')])

#df["id"] = df.index + 1

df = df[['gold_label','sentence1','sentence2']]

df = df.loc[df["gold_label"] != '-' ]

kf = KFold(n_splits=10, shuffle=False)

!mkdir folds

for i, (t_ind, v_ind) in enumerate(kf.split(df)):
    
    df.iloc[t_ind].to_csv(f'folds/snli_data_train_{i}.csv')   # train set
    df.iloc[v_ind].to_csv(f'folds/snli_data_valid_{i}.csv')   # validation set





In [13]:
#@title Modelo_1
class GCNBERTNLIModel(nn.Module):
    def __init__(self, bert_model, output_dim):
        super().__init__()
        self.bert = bert_model
        self.conv1 = GCNConv(768, 768)
        self.conv2 = GCNConv(768, 768)

        embedding_dim = bert_model.config.to_dict()['hidden_size']

        self.out = nn.Linear(768 * 512, 256)
        self.out2 = nn.Linear(256, output_dim)

    def forward(self, sequence, attn_mask, token_type, edge_index):
        embedded = self.bert(input_ids = sequence, attention_mask = attn_mask, token_type_ids= token_type)
        shape = embedded[0].shape
        x = self.conv1(torch.reshape(embedded[0],(shape[0] * shape[1],shape[2])), edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        #x = self.conv2(x, edge_index)
        x = torch.reshape(x, shape)
        x = x.view(x.size(0), -1)

        output = self.out(x)

        output = self.out2(output)

        return output



In [ ]:
#@title Loop de treinamento

gc.collect()
torch.cuda.empty_cache()

for f in range(N_FOLDS):
  train_data_len,  train_dataloader = getDataLoader(f, BATCH_SIZE, FOLDER, PREFIXO, 'train')
  valid_data_len,  valid_dataloader = getDataLoader(f, BATCH_SIZE, FOLDER, PREFIXO, 'valid')

  torch.manual_seed(42)
  bert_model = BertModel.from_pretrained('bert-base-uncased')
  model = GCNBERTNLIModel(bert_model, OUTPUT_DIM).to(device)
  torch.cuda.empty_cache() # releases all unoccupied cached memory

  optimizer = optim.AdamW(model.parameters(),lr=2e-5,eps=1e-6)
  total_steps = math.ceil(N_EPOCHS*train_data_len*1./BATCH_SIZE)
  warmup_steps = int(total_steps*warmup_percent)
  scheduler = get_scheduler(optimizer, warmup_steps)
  best_valid_loss = float('inf')
  print(f'Fold - {f}')

  for epoch in range(N_EPOCHS):
    gc.collect()
    start_time = time.time()
    train_loss, train_acc = treinarEpoca(model, train_dataloader, optimizer, criterion, scheduler)
    valid_loss, valid_acc = validarEpoca(model, valid_dataloader, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
      best_valid_loss = valid_loss
      torch.save(model.state_dict(), 'bert-nli.pt')
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f't Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
#@title Salvar/Carregar modelos

#from google.colab import files
#files.download('model.onnx') 
!cp model.onnx /content/drive/MyDrive/mestrado/model.onnx

#!cp ./bert-nli.pt /content/drive/MyDrive/mestrado/colab/modelosTreinados/bert_snli_BI_k04.pt
!cp /content/drive/MyDrive/mestrado/colab/modelosTreinados/bert_snli_BI_k02.pt .

In [ ]:
bert_model = BertModel.from_pretrained('bert-base-uncased')
model = GCNBERTNLIModel(bert_model, OUTPUT_DIM).to(device)
premise = 'john and his brother is wallking in the park'
hypothesis = 'john is has a sister'
inferir(premise, hypothesis, model, ['Y','N'])